<a href="https://colab.research.google.com/github/nagabathula/C255-DataMIning/blob/main/Line_Charts_with_D3_(NYC_Taxi_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preliminary Analysis

a) NYC Taxi Dataset 
b) Used D3 linecharts 

In [47]:
import pandas as pd

def load_data():
  return pd.concat([pd.read_parquet('/content/green_tripdata_2021-01.parquet')])

raw_data = load_data()

print(raw_data.head())
print(raw_data.info())

   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2021-01-01 00:15:56   2021-01-01 00:19:52                  N   
1         2  2021-01-01 00:25:59   2021-01-01 00:34:44                  N   
2         2  2021-01-01 00:45:57   2021-01-01 00:51:55                  N   
3         2  2020-12-31 23:57:51   2021-01-01 00:04:56                  N   
4         2  2021-01-01 00:16:36   2021-01-01 00:16:40                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0         1.0            43           151              1.0           1.01   
1         1.0           166           239              1.0           2.53   
2         1.0            41            42              1.0           1.12   
3         1.0           168            75              1.0           1.99   
4         2.0           265           265              3.0           0.00   

   fare_amount  extra  mta_tax  tip_amount  tolls_amount ehail_fee  \
0   

In [48]:
taxi_df = raw_data.copy()
taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               76518 non-null  int64         
 1   lpep_pickup_datetime   76518 non-null  datetime64[ns]
 2   lpep_dropoff_datetime  76518 non-null  datetime64[ns]
 3   store_and_fwd_flag     40471 non-null  object        
 4   RatecodeID             40471 non-null  float64       
 5   PULocationID           76518 non-null  int64         
 6   DOLocationID           76518 non-null  int64         
 7   passenger_count        40471 non-null  float64       
 8   trip_distance          76518 non-null  float64       
 9   fare_amount            76518 non-null  float64       
 10  extra                  76518 non-null  float64       
 11  mta_tax                76518 non-null  float64       
 12  tip_amount             76518 non-null  float64       
 13  t

In [49]:
taxi_df = raw_data.copy()

taxi_df['lpep_pickup_date'] = \
    pd.DatetimeIndex(taxi_df['lpep_pickup_datetime']).date

taxi_df['lpep_pickup_time'] = \
    pd.DatetimeIndex(taxi_df['lpep_pickup_datetime']).time

taxi_df = taxi_df[['VendorID', 'lpep_pickup_date', 
                   'lpep_pickup_time', 'PULocationID', 
                   'DOLocationID', 'total_amount',
                   'tip_amount']]

taxi_df.head()

,VendorID,lpep_pickup_date,lpep_pickup_time,PULocationID,DOLocationID,total_amount,tip_amount
0,2,2021-01-01,00:15:56,43,151,6.80,0.00
1,2,2021-01-01,00:25:59,166,239,16.86,2.81
2,2,2021-01-01,00:45:57,41,42,8.30,1.00
3,2,2020-12-31,23:57:51,168,75,9.30,0.00
4,2,2021-01-01,00:16:36,265,265,-52.80,0.00


In [50]:
taxi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   VendorID          76518 non-null  int64  
 1   lpep_pickup_date  76518 non-null  object 
 2   lpep_pickup_time  76518 non-null  object 
 3   PULocationID      76518 non-null  int64  
 4   DOLocationID      76518 non-null  int64  
 5   total_amount      76518 non-null  float64
 6   tip_amount        76518 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 4.1+ MB


## Group by `'lpep_pickup_date'` and `'VendorID'`

In [51]:
pd.DataFrame(taxi_df.groupby(['lpep_pickup_date','VendorID']).size()).head()

0
lpep_pickup_date VendorID     
2009-01-01       2           2
2020-12-31       2           1
2021-01-01       1          96
                 2         935
2021-01-02       1         174

In [52]:
pickups_by_date = \
    pd.DataFrame(taxi_df.groupby(['lpep_pickup_date','VendorID'])
                        .size())
  
pickups_by_date.reset_index(inplace=True)
pickups_by_date = \
    pickups_by_date.rename(
                  columns = {
                              'index':'lpep_pickup_date', 
                              0 : 'count', 
                              'VendorID' : 'vendor_id'
                            })

pickups_by_date.head(10)

,lpep_pickup_date,vendor_id,count
0,2009-01-01,2,2
1,2020-12-31,2,1
2,2021-01-01,1,96
3,2021-01-01,2,935
4,2021-01-02,1,174
5,2021-01-02,2,1730
6,2021-01-03,1,114
7,2021-01-03,2,1206
8,2021-01-04,1,236
9,2021-01-04,2,2672


In [53]:
pickups_by_date[pd.DatetimeIndex(pickups_by_date['lpep_pickup_date']).year != 2021]

,lpep_pickup_date,vendor_id,count
0,2009-01-01,2,2
1,2020-12-31,2,1


### Drop Outlier(s)

In [54]:
pickups_by_date[pickups_by_date['lpep_pickup_date'] == pd.Timestamp(2009, 1, 1)]
#pickups_by_date[pickups_by_date['lpep_pickup_date'] == pd.Timestamp(2020, 12, 31)]

/usr/local/lib/python3.7/dist-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,lpep_pickup_date,vendor_id,count
0,2009-01-01,2,2


In [55]:
pickups_by_date = \
    pickups_by_date[
      pd.DatetimeIndex(pickups_by_date['lpep_pickup_date']).year == 2021 ]

In [56]:
pickups_by_date.to_csv("pickup_counts_by_date.csv", index=False)

### Setup the http Server using python



In [57]:
from IPython.core.display import HTML
from string import Template
import json, random

In [58]:
from IPython.core.display import HTML
from string import Template
import json, random

# local server
port = 8888
get_ipython().system_raw('python3 -m http.server ' + str(port) + ' &') 

In [59]:
# Create template
html_page = """
  <link rel="stylesheet" href="https://localhost:$port/linecharts.css"/>
  <script src='https://d3js.org/d3.v7.min.js'></script>
  <script src="https://localhost:$port/linecharts.js"/>
"""

html_page = html_page.replace("$port", str(port)).strip()

In [60]:
%%writefile linecharts.css
path.line {
  fill: none;
  stroke-width: 2;
  color: #00ff00;
}

.axis text {
  font-family: Georgia;
  font-size: 9px;
  color: #00ff00;
}

.line_label {
  font-family: Georgia;
  font-size: 10px;
  color: #00ff00;
}

Overwriting linecharts.css


In [61]:
%%writefile linecharts.js
timeConv = d3.timeParse("%Y-%m-%d");
vendor_data_slices = { 1 : [], 2 : []}

csv_data = d3.csv("https://localhost:8888/pickup_counts_by_date.csv", function(row) { 
  row['lpep_pickup_date'] = timeConv(row['lpep_pickup_date'])
  row['count'] = parseInt(row['count'])

  if(row['vendor_id'] == 1) {
      vendor_data_slices[1].push(row);
  }
  else {
      vendor_data_slices[2].push(row);
  }
  return row;
})
csv_data.then(function(data) {

    console.log(data);
    console.log(vendor_data_slices);
 
});

Overwriting linecharts.js


In [62]:
display(HTML(html_page))

In [63]:
!sed -i '$ d' linecharts.js

In [64]:
%%writefile -a linecharts.js

    width = 1200; height = 600; margin = 20; padding = 140;

    var svg = d3.select("body")
                .append("svg")
                .attr("id", "svg")
                .attr("width", width)
                .attr("height", height);

    svg.text("NY Taxi Data - Simple Line Charts with D3").attr("id", "title")

    plot = svg.append("g").attr("id", "plot")

});

Appending to linecharts.js


In [65]:
display(HTML(html_page))

In [66]:
!sed -i '$ d' linecharts.js

In [67]:
%%writefile -a linecharts.js

    xScale = d3.scaleTime().range([0 + padding, width - padding]);
    yScale = d3.scaleLinear().rangeRound([height  - .5 * padding, 0 + .5 * padding]);

    xScale.domain(d3.extent(data, function(d){ return d.lpep_pickup_date; }));
    yScale.domain([(0), d3.max(data, function(d) { return d.count;}) ]);

    yaxis = d3.axisLeft().scale(yScale);

    // get the number of days for a given month
    no_days_in_month = new Date(2021, 1, 0). getDate();

    xaxis = d3.axisBottom()
                .ticks(no_days_in_month)
                .tickFormat(d3.timeFormat('%a %e'))
                .scale(xScale);

    x_axis = plot.append("g")
        .attr("id", "x-axis")
        .attr("class", "axis")
        .attr("transform", "translate(0," + (height - .5 * padding) + ")")
        .call(xaxis);

    y_axis = plot.append("g")
        .attr("id", "y-axis")
        .attr("class", "axis")
        .attr("transform", "translate(" + padding + ",0)")
        .call(yaxis);
});

Appending to linecharts.js


In [68]:
display(HTML(html_page))

## Axis Labels

In [69]:
!sed -i '$ d' linecharts.js

In [70]:
%%writefile -a linecharts.js
    x_axis.append("text")
        .attr("x", .5 * width)
        .attr("y", .35 * padding)
        .style("text-anchor", "end")
        .style("color", "red")
        .text("Day of Week & Month");

  y_axis.append("text")
        .attr("transform", "rotate(-90)")
        .attr("x", - .5 * height + .5 * padding)
        .attr("y", - .5 * padding)
        .style("color", "red")
        .text("Number of pickups by Vendor ID");
});

Appending to linecharts.js


In [71]:
display(HTML(html_page))

In [72]:
!sed -i '$ d' linecharts.js

In [73]:
%%writefile -a linecharts.js

   line = d3.line()
            .x(function(d) {
                return xScale(d.lpep_pickup_date);;
            })
            .y(function(d) {
                return yScale(d.count);
            });

    lines = plot.append("g").attr("id", "lines");

    var colorArray = [d3.schemeCategory10, d3.schemeAccent];
    var colorScheme = d3.scaleOrdinal(colorArray[0]);

    vendor_ids = [1,2]
    for (let idx in vendor_ids) {
        line_data = vendor_data_slices[vendor_ids[idx]]

        path = lines.append("path")
        path.datum(line_data)
            .attr("class", "line")
            .attr("d", line ).attr("stroke", colorScheme(idx));

        lines.append("text")
            .attr("class","line_label")
            .datum(function(d) {
                return {
                    vendor_id: vendor_ids[idx],
                    count: line_data[line_data.length - 1].count,
                    lpep_pickup_date: line_data[line_data.length - 1].lpep_pickup_date
                };})
            .attr("transform", function(d) {
                    console.log(d);
                    return "translate(" + (xScale(d.lpep_pickup_date) + 3)  + "," + (yScale(d.count) + 5 )+ ")"; })
            .attr("x", 5)
            .text(function(d) { return "Vendor ID: #" + d.vendor_id; })
            .attr("fill", colorScheme(idx));
    }
});

Appending to linecharts.js


In [74]:
display(HTML(html_page))